In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd

from CoolProp.CoolProp import PropsSI

import matplotlib.pyplot as plt
%matplotlib inline

cwd = os.getcwd()
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(os.path.normpath(os.path.join(cwd, '..', '..', '..', 'standalone')))

from standalone.run_g_function import RunGFunctions

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('MFRTRT_loads.csv', parse_dates=True, index_col='Date/Time')

In [ ]:
df.head(3)

In [ ]:
df = df.resample('1T').mean()
df = df.interpolate(method='linear')

In [ ]:
df.head(3)

In [ ]:
df.plot(y=['mdot [kg/s]'])

In [ ]:
df.plot(y=['Meas. Total Power [W]'])

In [ ]:
df.to_csv('in.csv', columns=['Meas. Total Power [W]', 'mdot [kg/s]'])

In [ ]:
in_file = os.path.join(os.getcwd(), 'MFRTRT_input.json')
with open(in_file, 'r') as f:
    json_blob = f.read()
in_json = json.loads(json_blob)

In [ ]:
load_file = in_json['load-profile']['external']['path'] 
in_json['load-profile']['external']['path'] = os.path.join(os.getcwd(), load_file)

flow_file = in_json['flow-profile']['external']['path']
in_json['flow-profile']['external']['path'] = os.path.join(os.getcwd(), flow_file)

g_file = in_json['g-functions']['file']
in_json['g-functions']['file'] = os.path.join(os.getcwd(), g_file)

In [ ]:
with open(os.path.join(os.getcwd(), 'in.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))

In [ ]:
in_file = os.path.join(os.getcwd(), 'in.json')

In [ ]:
g = RunGFunctions(in_file)

In [ ]:
g.simulate()

In [ ]:
df_model = pd.read_csv("out.csv")

In [ ]:
df_model.head(1)

In [ ]:
start_pt = 0
end_pt = 120

x = range(start_pt, end_pt)

in_model = df_model['GLHE Inlet Temperature [C]'].values[start_pt : end_pt]
out_model = df_model['GLHE Outlet Temperature [C]'].values[start_pt : end_pt]


plt.plot(x, in_model, label='Model Inlet', linestyle='--')
plt.plot(x, out_model, label='Model Outlet', linestyle='--')


in_exp = df['Inlet 1 [C]'].values[start_pt : end_pt]
out_exp = df['Outlet 1 [C]'].values[start_pt : end_pt]

plt.plot(x, in_exp, label='Exp. Inlet')
plt.plot(x, out_exp, label='Exp. Outlet')

plt.legend()

plt.show()

In [ ]:
plt.plot(x, df_model['Flow Fraction [-]'][start_pt : end_pt])
plt.show()

In [ ]:
plt.plot(x, df['Meas. Total Power [W]'].values[start_pt : end_pt], label="Exp Data")
plt.plot(x, df_model['GLHE Heat Transfer Rate [W]'].values[start_pt : end_pt], label="Model")

plt.legend()

plt.show()

In [ ]:
plt.plot(x, df_model['Load on GHE [W/m]'].values[start_pt : end_pt]) 

plt.show()

In [ ]:
plt.plot(x, df_model['Average Fluid Temp [C]'].values[start_pt : end_pt], label="Tf")
plt.legend()
plt.show()

In [ ]:
plt.plot(x, df_model["Local Borehole Resistance 'Rb' [K/(W/m)]"].values[start_pt : end_pt], label="Rb")
plt.plot(x, df_model["Soil Resistance 'Rs' [K/(W/m)]"].values[start_pt : end_pt], label="Rs")

plt.legend()
plt.show()